In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score, mean_squared_error


In [4]:
df_evo = pd.read_csv('csv/Evolution_DataSets.csv')

especies_de_interes = ['Homo Neanderthalensis\n', 'Homo Erectus', 'Homo Sapiens', 'Homo Habilis', 'Homo Heidelbergensis']
df_especies_de_interes = df_evo[df_evo['Specie'].isin(especies_de_interes)]

features = ['Cranial_Capacity', 'Height']
target = ['Specie']

X = df_especies_de_interes[features].values
y = df_especies_de_interes[target].values

print(X.shape) # Cantidad de muestras (filas) y características (columnas)
print(df_evo['Specie'].drop_duplicates()) # Especies en el df.

(2500, 2)
0                  hominino Orrorin tugenencin
1      hominino Ardipithecus ramidus / kabadda
2                   Australopithecus Afarensis
3                   Australopithecus Anamensis
4                   Australopithecus Africanus
5                            Homo Rodhesiensis
7           hominino Sahelanthropus tchadensis
8                      Homo Neanderthalensis\n
11                    Paranthropus Aethiopicus
12                                Homo Erectus
13                                 Homo Naledi
14                           Homo Floresiensis
16                         Paranthropus Boisei
18                            Homo Rudolfensis
20                                Homo Habilis
24                                Homo Sapiens
25                              Homo Antecesor
35                               Homo Ergaster
37                     Australopithecus Sediba
41                              Homo Georgicus
50              Australopithecus Bahrelghazali
52 

In [3]:
kernel = 1.0 * RBF([1.0])

gpc = GaussianProcessClassifier(kernel=kernel).fit(X, y.ravel())

# RBF es un modelo que intenta aprender la distribución de las clases en los datos de entrada (las características de las plantas) y clasificar nuevos puntos de datos en función de esa distribución aprendida.
# GaussianProcessClassifier es una clase en la biblioteca scikit-learn que implementa un clasificador basado en procesos gaussianos para problemas de clasificación.
print(kernel)

C:\sw\jupyter\jupyter_1\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\sw\jupyter\jupyter_1\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1**2 * RBF(length_scale=1)


In [ ]:
# Añadimos los límites de los ejes x, y en la futura gráfica. 
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1  # cogemos el valor mínimo y máximo de la primera característica (sepal lenght) y le restamos o sumamos 1.
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1  # cogemos el valor mínimo y máximo de la segunda característica (sepal width) y le restamos o sumamos 1.

h = 0.5  # Densidad de la "superficie" de la gráfica (o malla; cuanto más mayor sea, menor densidad)

# Meshgrid crea una malla bidimensionales (la matriz 'xx' tendrá las coordenadas x de todos los puntos y la matriz 'yy' tendrá las coordenadas de la y)
# np.arange se usa para generar los puntos de las coordenadas de las gráficas usando los límites que le pasamos (np.arange(start, stop, step) -> (stop - start / step)).
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))


In [ ]:
plt.figure(figsize=(10, 5))
    
# Se utiliza el clasificador actual (clf) para predecir las probabilidades para todos los puntos en la malla bidimensional ("aplanando" con .ravel).
Z = gpc.predict_proba(np.c_[xx.ravel(), yy.ravel()])   

# "Reformatea" el objeto Z, estableciendo sus tres dimensiones
Z = Z.reshape((xx.shape[0], xx.shape[1], 3))
print(xx.shape[0])  # el número de filas. Cada fila corresponde a una coordenada en a lo largo del eje y.
print(xx.shape[1])  # el número de columnas. Cada columna corresponde a una coordenada en a lo largo del eje x.
    
# Plotea la imagen con las probabilidades que se han establecido en la predicción de Z (ejecutar hasta esta línea sin tener en cuenta las siguientes para verlo más claro)
plt.imshow(Z, extent=(x_min, x_max, y_min, y_max), origin="lower")
    
# Hace un scatter con los datos de todas las filas (clases) y las dos primeras columnas (sepal lenght y sepal width). 
# Cada clase en [y] se colorea con su color correspondiente (red->0, green->1, blue->2).
plt.scatter(X[:, 0], X[:, 1], c=np.array(["r", "g", "b"])[y], edgecolors=(0, 0, 0))

plt.xlabel("Sepal length")
plt.ylabel("Sepal width")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

accuracy = accuracy_score(y, gpc.predict(X))
mse = mean_squared_error(y, gpc.predict(X))
# Con MSE tomamos la diferencia entre cada valor predicho y su valor real, luego elevamos la diferencia al cuadrado y calculamos el promedio de la diferencia al cuadrado.
    
plt.title(
    "%s, LML: %.3f\n Accuracy: %.3f\n MSE: %.3f" % (titles[i], gpc.log_marginal_likelihood(gpc.kernel_.theta), accuracy, mse) 
    # Una LML (Verosimilitud Marginal Logarítmica) más alta indica un mejor ajuste del modelo a los datos observados.
    # Sin embargo, los valores numéricos de la LML no son inherentemente significativos. La importancia de estos valores radica en la comparación relativa entre diferentes modelos o configuraciones de modelo.
)
plt.tight_layout()
plt.show()
    

<span style="font-size: 18px;">

GaussianProcessClassifier:

Modelado Probabilístico: El clasificador utiliza un modelo probabilístico para asignar una probabilidad a cada clase para una instancia de entrada dada. En lugar de solo predecir una clase, proporciona una distribución de probabilidad sobre todas las clases posibles.

Kernel de Covarianza: Utiliza un kernel de covarianza para modelar la relación entre las observaciones en el espacio de características. El kernel de covarianza es fundamental en los procesos gaussianos ya que determina cómo las observaciones se relacionan entre sí en función de sus características.

Aprendizaje Bayesiano: Se basa en el aprendizaje bayesiano para ajustar los hiperparámetros del modelo y realizar inferencias sobre los datos de entrenamiento. Esto significa que no solo proporciona predicciones puntuales, sino que también proporciona una medida de incertidumbre asociada con cada predicción.

Flexibilidad: Los procesos gaussianos son modelos no paramétricos, lo que significa que no hacen suposiciones específicas sobre la forma funcional de la relación entre las características y las etiquetas. Esto les otorga una gran flexibilidad para modelar relaciones complejas en los datos.
</span>